# Kenya
This notebook is used to create the dataset for Kenya. The dataset consists of multiple files. 
- MTF_HH_Roster : This file contains the information about the household.
- MTF_HH_Cooking_Data_Final : This file contains the information about the cooking data of the household.

In [1]:
import os
import sys
import pandas as pd

sys.path.append("../")  # Adds higher directory to python modules path.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from core.ODEDataset import ODEDataset
from utils import common_modifiers, dwelling_modifiers, socio_modifiers, finance_modifiers
from utils import constants



# Clusters
Clusters are group of columns used together to create a new feature. 

### Cooking Hours Cluster
- i_23_hrs_morning
- i_24_hrs_afternoon
- i_25_hrs_evening
- i_26_hrs_h2o

In [2]:
cooking_hrs_cluster = [
    "i_23_hrs_morning",
    "i_24_hrs_afternoon",
    'i_25_hrs_evening',
    'i_26_hrs_h2o'
]

monthly_expenditure_cluster = ['l_l_12',
                               'l_l_13',
                               'l_l_14',
                               'l_l_15',
                               'l_l_16',
                               'l_l_17',
                               'l_l_18',
                               'l_l_19']


# Constants


# Categories
Map the original values to the standard values.
- Main Occupation
- Education Level
- Dwelling Wall Quality
- Dwelling Roof Quality
- Dwelling Floor Quality
- Dwelling Toilet Quality
- Dwelling Water Quality
-  
 

In [3]:
Main_occupation_original2final = {
    'Unemployed': 'Unemployed',
    'Student': 'Unemployed',
    'Wage Employee, Non-Farm': 'Employee non-farm',
    'Self-Employed Agriculture/Livestock': 'Own-account worker farm',
    'Casual/Day Laborer': 'Worker not classifiable by status',
    'Self-Employed Non-Farm-Business enterprise': 'Own-account worker non-farm',
    'Job Seeker': 'Unemployed',
    'Wage Employee, Farm': 'Employee farm',
    'Too old to work': 'Unemployed',
    'Self-Employed Non-Farm-Independent contractor,technician, professional, etc.': 'Own-account worker non-farm',
    'Retired/pensioner': 'Unemployed',
    'Assistance in family enterprise': 'Contributing family worker',
    'Other': 'Worker not classifiable by status',
    'Disabled': 'Unemployed',
    'Intern/free labor/voluntary work': 'Worker not classifiable by status'
}

Education_level_original2final = {
    'None': 'No schooling',
    'Primary': 'Primary education',
    'Secondary School': 'Lower secondary education',
    'Vocational/Technical/School- after completing primary education': 'Post-secondary non-tertiary education',
    'Vocational/Technical/School- after completing secondary education': 'Post-secondary non-tertiary education',
    'University': "Bachelor's or equivalent level",
    'Post-Grad': "Doctoral or equivalent level",
    'Other': 'Not elsewhere classified'
}

Dwelling_Wall_Quality_label2id = {
    'Blocks, plastered with cement': 1,
    'Stone and cement': 1,
    'Blocks, unplastered': 1,
    'Steel': 1,
    'Bricks': 1,
    'Mud bricks (traditional)': 0,
    'Wood and mud': 0,
    'Corrugated iron sheet': 0,
    'Chip wood': 0,
    'Parquet or polished wood': 0,
    'Wood only': 0,
    'Stone only': 0,
    'Wood and thatch': 0,
    'Stone and mud': 0,
    '0': 0,
    'Reed/bamboo': 0,
    'Asbestos': 0,
    'Other': 0,
    'Cargo container': 0
}

Dwelling_Roof_Quality_label2id = {
    'Corrugated iron sheet': 0,
    'Thatch': 0,
    'Concrete/Cement': 1,
    'Wood and mud': 0,
    'Bricks': 1,
    'Asbestos': 0,
    'Bamboo/Reed': 0,
    '0': 0,
    'Plastic canvas': 0,
    'Other': 0
}

Dwelling_Floor_Quality_label2id = {
    'Mud/Dung': 0,
    'Cement tiles': 1,
    'Ceramic/Marble tiles': 1,
    'Brick tiles': 1,
    'Plastic tiles': 1,
    'Wood planks': 1,
    'Parquet or polished wood': 1,
    'Cement screed': 1,
    '0': 0,
    'Other': 0,
    'Bamboo/Reed': 0
}

Dwelling_Toilet_Quality_label2id = {
    'Covered pit latrine': 1,
    'Uncovered pit latrine': 0,
    'Flush to sewage': 1,
    'Flush to septic tank': 1,
    'None': 1,
    'Community latrine': 0,
    'Pail / Bucket': 0,
    'Other, specify': 0,
    7.0: 0
}

Dwelling_Water_Quality_label2id = {
    'Piped water': 1,
    'Bore hole/ well': 1,
    'River/spring': 0,
    'Tanker/truck/vendor': 1,
    'Rain water': 1,
    'Lake/reservoir': 0,
    'Bottle water': 1,
    'Other': 0
}

Ownership_small_livestock_original2final = {
    'Camel': 0,
    'Cow/bull/calves': 0,
    'Goat': 1,
    'Horse/donkey': 0,
    'Other Animal': 0,
    'Pig': 1,
    'Sheep': 1
}

Ownership_large_livestock_original2final = {
    'Camel': 1,
    'Cow/bull/calves': 1,
    'Goat': 0,
    'Horse/donkey': 1,
    'Other Animal': 0,
    'Pig': 0,
    'Sheep': 0
}

Ownership_motorized_vehicle_original2final = {
    'Vehicle (Car, pickup truck, etc)': 1,
    'Bicycle': 0,
    'Domestic water pump': 0,
    'Motorcycle': 1,
    'Other equipment': 0,
    'Tractor': 0
}



In [4]:

# MTF_HH_Roster
MTF_HH_Roster = ODEDataset("keyna/MTF_HH_Roster")
MTF_HH_Roster.from_csv("../playground/data/ESMAP/kenya/MTF_HH_Roster.csv").group_by("cluuq")

MTF_HH_Roster = MTF_HH_Roster.new_feature("Age_HHH",
                                          socio_modifiers.extract_age_of_head('a_5_age', 'a_4_rel_hhh', 'Head'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Main_occupation',
                                          common_modifiers.categorize("a_14_main_occ", Main_occupation_original2final))

MTF_HH_Roster = MTF_HH_Roster.new_feature("HH_with_home_business",
                                          socio_modifiers.extract_head_w_home_business('a_19_hh_ent', 'Main_occupation',
                                                                                       'a_5_age', 'Yes'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Education_level_HHH',
                                          common_modifiers.categorize('a_9a_sch_lev', Education_level_original2final))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Number_workers', socio_modifiers.extract_working_people('Main_occupation'))
MTF_HH_Roster = MTF_HH_Roster.new_feature('Socio_status_HHH',
                                          socio_modifiers.extract_socio_status_hhh('Kenya', 'Main_occupation',
                                                                                   'a_4_rel_hhh',
                                                                                   'Head'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Number_adults', socio_modifiers.extract_age_groups('a_5_age', 'adults'))

In [5]:
MTF_HH_Roster = MTF_HH_Roster.select(
    ['cluuq', 'Age_HHH', 'Number_workers', 'Socio_status_HHH', 'Number_adults'])

MTF_HH_Roster.to_dataframe().head()

,cluuq,Age_HHH,Number_workers,Socio_status_HHH,Number_adults
0,0,50.0,NaN,Employee non-farm,15.0
1,1001_3,NaN,NaN,NaN,34.0
2,100_13,NaN,NaN,NaN,10.0
3,1026_3,NaN,NaN,NaN,59.0
4,1027_2,NaN,NaN,NaN,57.0


# MTF_HH_Cooking_Data_Final

In [6]:

MTF_HH_Cooking_Data_Final = ODEDataset("keyna/MTF_HH_Cooking_Data_Final")
MTF_HH_Cooking_Data_Final.from_csv("../playground/data/ESMAP/kenya/MTF_HH_Cooking_Data_Final.csv",
                                   encoding='ISO-8859-1').group_by("cluuq")

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.new_feature('Fuel_usage',
                                                                  dwelling_modifiers.extract_fuel_usage(
                                                                      cooking_hrs_cluster))

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.new_feature('Clean_fuel',
                                                                  dwelling_modifiers.is_clean_fuel(
                                                                      constants.CLEAN_FUELS,
                                                                      'i_18_a_1st_fuel'))

MTF_HH_Cooking_Data_Final.to_dataframe().head()

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.select(['cluuq', 'Clean_fuel'])


In [7]:
Kenya = MTF_HH_Roster.merge(MTF_HH_Cooking_Data_Final, 'cluuq')
Kenya.to_dataframe().head()

,cluuq,Age_HHH,Number_workers,Socio_status_HHH,Number_adults,Clean_fuel
0,0,50.0,NaN,Employee non-farm,15.0,No
1,1001_3,NaN,NaN,NaN,34.0,No
2,1026_3,NaN,NaN,NaN,59.0,No
3,1027_2,NaN,NaN,NaN,57.0,No
4,1029_3,NaN,NaN,NaN,29.0,No


## Core Survey

In [8]:
MTF_HH_Core_Survey = ODEDataset("keyna/MTF_HH_Core_Survey")
MTF_HH_Core_Survey.from_csv("../playground/data/ESMAP/kenya/MTF_HH_Core_Survey.csv", encoding='ISO-8859-1').group_by(
    "cluuq")

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature('Dwelling_wall',
                                                    common_modifiers.categorize('b_b_10',
                                                                                Dwelling_Wall_Quality_label2id))
MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Dwelling_roof",
                                                    common_modifiers.categorize("b_b_11",
                                                                                Dwelling_Roof_Quality_label2id))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Dwelling_floor",
                                                    common_modifiers.categorize("b_b12",
                                                                                Dwelling_Floor_Quality_label2id))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Dwelling_water",
                                                    common_modifiers.categorize("b_b_14",
                                                                                Dwelling_Water_Quality_label2id))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Dwelling_toilet",
                                                    dwelling_modifiers.get_dwelling_toilet("b_b_13A", "b_b_13B",
                                                                                           "b_b_13C",
                                                                                           Dwelling_Toilet_Quality_label2id))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Dwelling_quality_index", dwelling_modifiers.dwelling_quality_index)

MTF_HH_Core_Survey.to_dataframe().head()[
    ['cluuq', 'Dwelling_wall', 'Dwelling_roof', 'Dwelling_floor', 'Dwelling_water', 'Dwelling_toilet',
     'Dwelling_quality_index']]


,cluuq,Dwelling_wall,Dwelling_roof,Dwelling_floor,Dwelling_water,Dwelling_toilet,Dwelling_quality_index
0,1001_3,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",[1],0.6
1,100_13,"[1, 1, 1, 1, 1]","[1, 0, 0, 0, 0]","[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]",[1],1.0
2,1026_3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]",[1],0.4
3,1027_2,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]",[1],0.2
4,1029_3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1]",[1],0.4


In Order to get the hours available electricity, we need to get the connection type and data of the household from the MTF_HH_Solar_Roster file.

In [9]:
MTF_HH_Solar_Roster = ODEDataset('kenya/MTF_HH_Solar_Roster').from_csv(
    "../playground/data/ESMAP/kenya/MTF_HH_Solar_Roster.csv", encoding='ISO-8859-1')

MTF_HH_Core_Survey = MTF_HH_Core_Survey.merge(MTF_HH_Solar_Roster, on='cluuq', how='left')

connection_type_valid_answers = {constants.NATIONAL_GRID: 'Yes', constants.LOCAL_MINI_GRID: 'Yes',
                                 constants.SOLAR_HOME_SYSTEM: 1}
connection_type_modifier = dwelling_modifiers.get_connection_type('c_c_2',
                                                                  'c_c_38', 'c_c_q122_3',
                                                                  connection_type_valid_answers)

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature("Connection_type", connection_type_modifier)

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature(
    "Hours_available_electricity",
    dwelling_modifiers.get_hours_available_electricity('c_c_25aii', 'c_c_25aii', 'c_127_device',
                                                       'c_c_147b_Typicalmonth'))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature('Measurement_age',
                                                    socio_modifiers.measurement_age('c_c_7', 'c_c_38', 'c_135_yrs',
                                                                                    'c_127_device'))

MTF_HH_Core_Survey = MTF_HH_Core_Survey.new_feature('Monthly_expenditure',
                                                    finance_modifiers.calculate_expenditure_monthly(
                                                        monthly_expenditure_cluster))

In [10]:
MTF_HH_Core_Survey = MTF_HH_Core_Survey.select(['cluuq',
                                                'Dwelling_quality_index',
                                                'Hours_available_electricity',
                                                'Measurement_age',
                                                'Monthly_expenditure',
                                                ])


## Asset_Data_Final
Using the M1_Asset_Data_Final and M2_Asset_Data_Final files, we can get the information about the assets of the household.

In [18]:
M1_Asset_Data_Final = ODEDataset('kenya/MTF_HH_Sec.M1_Asset_Data_Final').from_csv(
    "../playground/data/ESMAP/kenya/MTF_HH_Sec.M1_Asset_Data_Final.csv", encoding='ISO-8859-1')

M1_Asset_Data_Final = M1_Asset_Data_Final.group_by("cluuq")
M1_Asset_Data_Final = M1_Asset_Data_Final.new_feature("Ownership_motorized_vehicle_all",
                                                      common_modifiers.categorize('m_a_label',
                                                                                  Ownership_motorized_vehicle_original2final,
                                                                                  ))


def get_ownership_vehicle(row: pd.Series):
    if any(t == 1 for t in row['Ownership_motorized_vehicle_all']):
        result = 'Yes'
    else:
        result = 'No'
    return result


M1_Asset_Data_Final = M1_Asset_Data_Final.new_feature("Ownership_motorized_vehicle",
                                                      common_modifiers.find_any('Ownership_motorized_vehicle_all', 1,
                                                                                'Yes', 'No'))



In [19]:
M1_Asset_Data_Final.to_dataframe().head()

,cluuq,HH_code,prov,cty,dist,div,loc,subloc,class,locality_urp,...,m_a_types,m_a_label,m_1_other,m_1_a,m_1_b_source,m_1_b_source_other,PARENT_KEY,KEY,Ownership_motorized_vehicle_all,Ownership_motorized_vehicle
0,0,[1090105011_23],[Coast],[Kilifi],[Kilifi],[Bahari],[Township],[Hospital],[Core],[Rural],...,[Motorcycle],[Motorcycle],[nan],[1.0],[11.0],[nan],[uuid:f86da9ca-8ecb-44a5-beca-a69b2bdcd352],[uuid:f86da9ca-8ecb-44a5-beca-a69b2bdcd352/m-m...,[1],Yes
1,1001_3,"[1090202021_09, 1090202021_10, 1090202021_01, ...","[Coast, Coast, Coast, Coast, Coast]","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi]","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi]","[Kikambala, Kikambala, Kikambala, Kikambala, K...","[Junju, Junju, Junju, Junju, Junju]","[Kuruwitu, Kuruwitu, Kuruwitu, Kuruwitu, Kuruw...","[Core, Core, Core, Core, Core]","[Rural, Rural, Rural, Rural, Rural]",...,"[Other equipment, Other equipment, Bicycle, Bi...","[Other equipment, Other equipment, Bicycle, Bi...","[10, 9, nan, nan, 9]","[1.0, 1.0, 1.0, 0.0, 3.0]","[13.0, 13.0, 13.0, 13.0, 13.0]","[nan, nan, nan, nan, nan]","[uuid:14ca78d4-a486-4a79-8613-facfb5eb36c5, uu...",[uuid:14ca78d4-a486-4a79-8613-facfb5eb36c5/m-m...,"[0, 0, 0, 0, 0]",No
2,1026_3,"[1090402012_07, 1090402012_05, 1090402012_08]","[Coast, Coast, Coast]","[Kilifi, Kilifi, Kilifi]","[Kilifi, Kilifi, Kilifi]","[Ganze, Ganze, Ganze]","[Palakumi, Palakumi, Palakumi]","[Palakumi, Palakumi, Palakumi]","[Core, Core, Core]","[Rural, Rural, Rural]",...,"[Bicycle, Bicycle, Motorcycle]","[Bicycle, Bicycle, Motorcycle]","[nan, nan, nan]","[2.0, 1.0, 2.0]","[13.0, 13.0, 11.0]","[nan, nan, nan]","[uuid:767bd398-e5a0-423e-96a3-c662454484a5, uu...",[uuid:767bd398-e5a0-423e-96a3-c662454484a5/m-m...,"[0, 0, 1]",Yes
3,1027_2,"[1090402021_08, 1090402021_12]","[Coast, Coast]","[Kilifi, Kilifi]","[Kilifi, Kilifi]","[Ganze, Ganze]","[Palakumi, Palakumi]","[Mariakani/Vitsapuni, Mariakani/Vitsapuni]","[Core, Core]","[Rural, Rural]",...,"[Bicycle, Motorcycle]","[Bicycle, Motorcycle]","[nan, nan]","[1.0, 1.0]","[13.0, 11.0]","[nan, nan]","[uuid:0f24cf78-de6a-4e60-b1dc-0264b524d4fa, uu...",[uuid:0f24cf78-de6a-4e60-b1dc-0264b524d4fa/m-m...,"[0, 1]",Yes
4,1029_3,"[1090403021_03, 1090403021_12]","[Coast, Coast]","[Kilifi, Kilifi]","[Kilifi, Kilifi]","[Ganze, Ganze]","[Dungicha, Dungicha]","[Dungicha, Dungicha]","[Core, Core]","[Rural, Rural]",...,"[Bicycle, Bicycle]","[Bicycle, Bicycle]","[nan, nan]","[1.0, 1.0]","[13.0, 13.0]","[nan, nan]","[uuid:4c7d446b-88fb-4540-b98c-0293b15af58c, uu...",[uuid:4c7d446b-88fb-4540-b98c-0293b15af58c/m-m...,"[0, 0]",No


In [20]:
M2_Asset_Data_Final = ODEDataset('kenya/MTF_HH_Sec.M2_Asset_Data_Final').from_csv(
    "../playground/data/ESMAP/kenya/MTF_HH_Sec.M2_Asset_Data_Final.csv", encoding='ISO-8859-1')

M2_Asset_Data_Final = M2_Asset_Data_Final.group_by("cluuq")

M2_Asset_Data_Final = M2_Asset_Data_Final.new_feature("Ownership_small_livestock_all",
                                                      common_modifiers.categorize("m_a_label",
                                                                                  Ownership_small_livestock_original2final,
                                                                                  ))

M2_Asset_Data_Final = M2_Asset_Data_Final.new_feature("Ownership_large_livestock_all",
                                                      common_modifiers.categorize('m_a_label',
                                                                                  Ownership_large_livestock_original2final,
                                                                                  ))

M2_Asset_Data_Final = M2_Asset_Data_Final.new_feature("Ownership_small_livestock",
                                                      common_modifiers.find_any('Ownership_small_livestock_all', 1,
                                                                                'Yes', 'No'))

M2_Asset_Data_Final = M2_Asset_Data_Final.new_feature("Ownership_large_livestock",
                                                      common_modifiers.find_any('Ownership_large_livestock_all', 1,
                                                                                'Yes', 'No'))





In [21]:
M2_Asset_Data_Final.to_dataframe().head()

,cluuq,HH_code,prov,cty,dist,div,loc,subloc,class,locality_urp,...,m_b_types,m_a_label,m_2_other,m_2_a,PARENT_KEY,KEY,Ownership_small_livestock_all,Ownership_large_livestock_all,Ownership_small_livestock,Ownership_large_livestock
0,1001_3,"[1090202021_05, 1090202021_01, 1090202021_02, ...","[Coast, Coast, Coast, Coast]","[Kilifi, Kilifi, Kilifi, Kilifi]","[Kilifi, Kilifi, Kilifi, Kilifi]","[Kikambala, Kikambala, Kikambala, Kikambala]","[Junju, Junju, Junju, Junju]","[Kuruwitu, Kuruwitu, Kuruwitu, Kuruwitu]","[Core, Core, Core, Core]","[Rural, Rural, Rural, Rural]",...,"[Goat, Goat, Goat, Goat]","[Goat, Goat, Goat, Goat]","[nan, nan, nan, nan]","[6.0, 3.0, 6.0, 2.0]","[uuid:57b2afe2-a7bc-4f72-a28d-18694e1d825f, uu...",[uuid:57b2afe2-a7bc-4f72-a28d-18694e1d825f/m-m...,"[1, 1, 1, 1]","[0, 0, 0, 0]",Yes,No
1,1026_3,"[1090402012_06, 1090402012_02, 1090402012_02, ...","[Coast, Coast, Coast, Coast, Coast, Coast, Coa...","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi, Kilif...","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi, Kilif...","[Ganze, Ganze, Ganze, Ganze, Ganze, Ganze, Gan...","[Palakumi, Palakumi, Palakumi, Palakumi, Palak...","[Palakumi, Palakumi, Palakumi, Palakumi, Palak...","[Core, Core, Core, Core, Core, Core, Core, Cor...","[Rural, Rural, Rural, Rural, Rural, Rural, Rur...",...,"[Goat, Cow/bull/calves, Sheep, Goat, Goat, Cow...","[Goat, Cow/bull/calves, Sheep, Goat, Goat, Cow...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[5.0, 10.0, 6.0, 9.0, 6.0, 6.0, 13.0, 35.0, 60...","[uuid:3cb79a20-cb76-4902-b98b-c0007a452f5d, uu...",[uuid:3cb79a20-cb76-4902-b98b-c0007a452f5d/m-m...,"[1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...","[0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",Yes,Yes
2,1027_2,"[1090402021_08, 1090402021_06, 1090402021_07, ...","[Coast, Coast, Coast, Coast, Coast, Coast, Coa...","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi, Kilif...","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi, Kilif...","[Ganze, Ganze, Ganze, Ganze, Ganze, Ganze, Gan...","[Palakumi, Palakumi, Palakumi, Palakumi, Palak...","[Mariakani/Vitsapuni, Mariakani/Vitsapuni, Mar...","[Core, Core, Core, Core, Core, Core, Core, Cor...","[Rural, Rural, Rural, Rural, Rural, Rural, Rur...",...,"[Goat, Goat, Cow/bull/calves, Goat, Goat, Cow/...","[Goat, Goat, Cow/bull/calves, Goat, Goat, Cow/...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[3.0, 12.0, 1.0, 3.0, 6.0, 58.0, 24.0, 90.0, 6...","[uuid:0f24cf78-de6a-4e60-b1dc-0264b524d4fa, uu...",[uuid:0f24cf78-de6a-4e60-b1dc-0264b524d4fa/m-m...,"[1, 1, 0, 1, 1, 0, 1, 1, 0, 1]","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0]",Yes,Yes
3,1029_3,[1090403021_10],[Coast],[Kilifi],[Kilifi],[Ganze],[Dungicha],[Dungicha],[Core],[Rural],...,[Cow/bull/calves],[Cow/bull/calves],[nan],[6.0],[uuid:67db46be-adc7-4d5c-b5ef-fc6572423e06],[uuid:67db46be-adc7-4d5c-b5ef-fc6572423e06/m-m...,[0],[1],No,Yes
4,1037_5,"[1090903011_02, 1090903011_06, 1090903011_06, ...","[Coast, Coast, Coast, Coast, Coast, Coast, Coa...","[Kilifi, Kilifi, Kilifi, Kilifi, Kilifi, Kilif...","[Kaloleni, Kaloleni, Kaloleni, Kaloleni, Kalol...","[Kaloleni, Kaloleni, Kaloleni, Kaloleni, Kalol...","[Kambe, Kambe, Kambe, Kambe, Kambe, Kambe, Kam...","[Pangani/Maereni, Pangani/Maereni, Pangani/Mae...","[Core, Core, Core, Core, Core, Core, Core, Cor...","[Rural, Rural, Rural, Rural, Rural, Rural, Rur...",...,"[Goat, Sheep, Goat, Goat, Goat, Goat, Cow/bull...","[Goat, Sheep, Goat, Goat, Goat, Goat, Cow/bull...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[3.0, 1.0, 2.0, 5.0, 3.0, 7.0, 10.0, 25.0, 12....","[uuid:2a251377-d9fa-4eac-9f54-6056d6707a04, uu...",[uuid:2a251377-d9fa-4eac-9f54-6056d6707a04/m-m...,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",Yes,Yes


In [22]:
Asset_Data_Final = M1_Asset_Data_Final.merge(M2_Asset_Data_Final, on="cluuq")
Asset_Data_Final = Asset_Data_Final.select(['cluuq', 'Ownership_motorized_vehicle', 'Ownership_small_livestock', 'Ownership_motorized_vehicle'])

In [23]:
Asset_Data_Final.to_dataframe().head()

,cluuq,Ownership_motorized_vehicle,Ownership_small_livestock,Ownership_motorized_vehicle
0,1001_3,No,Yes,No
1,1026_3,Yes,Yes,Yes
2,1027_2,Yes,Yes,Yes
3,1029_3,No,No,No
4,1037_5,Yes,Yes,Yes
